In [1]:
import numpy as np
import tensorflow as tf

In [12]:
keyframes = tf.Variable(np.load("initial_trajectory.npy").reshape((-1, 7)), dtype=tf.float32)

In [3]:
start = tf.constant([1.548837512, 0.07374419285, -0.009645789198, -2.307796117, 0.03278595457, 2.367094903, 0.7636861643], dtype=tf.float32)
goal = tf.constant([0.5892901589, 0.4606513663, 0.02096505987, -2.409281471, -1.05241235, 1.91614986, 0.5328634008], dtype=tf.float32)

In [4]:
blue =    tf.constant([0.5892901589, 0.4606513663, 0.02096505987, -2.409281471, -1.05241235, 1.91614986, 0.5328634008], dtype=tf.float32)
green =   tf.constant([0.04828069357, 0.321198491, -0.09149035608, -2.445509451, 0.0339262541, 2.766537249, -0.7381757918], dtype=tf.float32)
red =     tf.constant([0.4973887075, 0.3626231152, -0.1262784644, -2.353311054, 0.1667072496, 2.697354126, -0.9114452175], dtype=tf.float32)

In [5]:
@tf.function
def integrate(a, b, function, num_samples=100):
    # from https://en.wikipedia.org/wiki/Numerical_integration
    positions = tf.cast(a + tf.range(num_samples+1)*(b-a)/num_samples, tf.float32)
    values = function(positions)
    integral = (b-a)/num_samples * (values[0]/2 + tf.reduce_sum(values[1:-1], axis=0) + values[-1]/2)
    return integral

In [8]:
@tf.function
def trajectory(t, keyframes):
    N = tf.constant(keyframes.shape[-2], dtype=np.float32)
    direction = tf.concat((keyframes, keyframes[-1, ...][None, ...]), axis=0)[1:, ...] - keyframes
    idx = tf.cast(tf.floor(t * (N-1)), tf.int64)
    time_shape = (*t.shape, *(1 for _ in range(len(keyframes.shape)-1)))
    return tf.gather(direction, idx, axis=0) * tf.reshape(tf.math.floormod(t*N, 1), time_shape) + tf.gather(keyframes, idx, axis=0)

@tf.function
def velocity(t, keyframes):
    N = t.shape[0]
    with tf.GradientTape() as tape:
        tape.watch(t)
        trajectory_points = trajectory(t, keyframes)
    jacobian = tape.jacobian(trajectory_points, t)
    vel = tf.reduce_sum(tf.reshape(jacobian, (*trajectory_points.shape, -1)), axis=-1)
    return vel

In [13]:
@tf.function
def cost(keyframes):
    return integrate(
        0, 1, 
        lambda t: tf.reduce_sum(velocity(t, keyframes)*velocity(t, keyframes), axis=-1)
    )

@tf.function
def constrain_start_goal(keyframes, start, goal):
    return tf.concat((start[None, :], keyframes[1:-1], goal[None, :]), 0)

def V(start, goal, *, num_keys=10, steps=10):
    num_keys = 10
    steps=10
    keyframes = tf.Variable(
        tf.linspace(start, goal, num_keys, axis=0), 
        constraint=lambda x: constrain_start_goal(x, start, goal)
        )
    opt = tf.optimizers.Adam(learning_rate=0.01)
    for idx in range(steps):
        opt.minimize(lambda: cost(keyframes), var_list=[keyframes])

    return cost(keyframes)

@tf.function
def V_fast(start, goal):
    return cost(tf.linspace(start, goal, 3, axis=0))

In [15]:
keyframes = tf.stack((keyframes,keyframes), axis=1)

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([0.10063807, 0.10063807], dtype=float32)>

In [32]:
cost(keyframes)

InvalidArgumentError:  indices[60] = 3 is not in [0, 3)
	 [[{{node PartitionedCall/PartitionedCall/PartitionedCall/PartitionedCall/GatherV2}}]] [Op:__inference_V_fast_3266]

Function call stack:
V_fast


In [8]:
keyframes

In [16]:
V_fast(current_pos, goal)

In [20]:
@tf.function
def P_goals(goal, num_goals=3):
    return 1/num_goals

@tf.function
def prop_goal(t, start, goal, keyframes, P_goals):
    current_pos = trajectory(t, keyframes)
    numerator = tf.math.exp(V_fast(start, goal) - V_fast(current_pos, goal))
    denominator = ( 
        tf.math.exp(V_fast(start, red) - V_fast(current_pos, red)) + 
        tf.math.exp(V_fast(start, blue) - V_fast(current_pos, blue)) + 
        tf.math.exp(V_fast(start, green) - V_fast(current_pos, green))
    )
    return numerator / denominator * P_goals(goal)

@tf.function
def normalization(t, T=1):
    return T - t

@tf.function
def legibility(start, goal, keyframes):
    numerator = integrate(0, 1, lambda t: prop_goal(t, start, goal, keyframes, P_goals)*normalization(t))
    denominator = integrate(0, 1, lambda t: normalization(t))
    return numerator / denominator

<tf.Tensor: shape=(4, 7), dtype=float32, numpy=
array([[ 1.5171725e+00,  8.6512126e-02, -8.6356308e-03, -2.3111451e+00,
        -3.0255876e-03,  2.3522139e+00,  7.5606900e-01],
       [ 1.3377371e+00,  1.5886377e-01, -2.9114021e-03, -2.3301229e+00,
        -2.0595768e-01,  2.2678871e+00,  7.1290517e-01],
       [ 1.2321869e+00,  2.0142356e-01,  4.5579160e-04, -2.3412864e+00,
        -3.2532951e-01,  2.2182829e+00,  6.8751466e-01],
       [ 1.1266366e+00,  2.4398336e-01,  3.8229846e-03, -2.3524497e+00,
        -4.4470131e-01,  2.1686790e+00,  6.6212416e-01]], dtype=float32)>

In [18]:
t = tf.convert_to_tensor([0.03, 0.2, 0.3, 0.4])

ValueError: in user code:

    <ipython-input-7-9556645fc166>:28 V_fast  *
        return cost(keyframes)
    <ipython-input-7-9556645fc166>:5 cost  *
        lambda t: tf.reduce_sum(velocity(t, keyframes)*velocity(t, keyframes), axis=-1)
    <ipython-input-5-a16a483bf719>:5 integrate  *
        values = function(positions)
    <ipython-input-6-2c8366097378>:13 velocity  *
        trajectory_points = trajectory(t, keyframes)
    <ipython-input-6-2c8366097378>:6 trajectory  *
        return tf.gather(direction, idx, axis=0) * tf.math.floormod(t*N, 1)[..., None] + tf.gather(keyframes, idx, axis=0)
    C:\Users\Sebastian\AppData\Local\Programs\Python\Python37\lib\site-packages\tensorflow\python\ops\math_ops.py:1180 binary_op_wrapper
        raise e
    C:\Users\Sebastian\AppData\Local\Programs\Python\Python37\lib\site-packages\tensorflow\python\ops\math_ops.py:1164 binary_op_wrapper
        return func(x, y, name=name)
    C:\Users\Sebastian\AppData\Local\Programs\Python\Python37\lib\site

In [ ]:
current_pos

In [173]:
current_pos = trajectory(t, keyframes)
numerator = tf.math.exp(V_fast(start, goal) - V_fast(current_pos, goal))
denominator = ( 
    tf.math.exp(V_fast(start, red) - V_fast(current_pos, red)) + 
    tf.math.exp(V_fast(start, blue) - V_fast(current_pos, blue)) + 
    tf.math.exp(V_fast(start, green) - V_fast(current_pos, green))
)

In [ ]:
prop_goal(t, start, goal, keyframes, P_goals)

In [ ]:
integrate(0, 1, lambda t: prop_goal(t, start, goal, keyframes, P_goals)*normalization(t))